# Cookbook: Drive Guide

In this cookbook, we will demonstrate how to query the distance between two locations and create a driving plan using an agent.

# Install Requirements

We will use [Qwen-Agent](https://github.com/QwenLM/Qwen-Agent) in this book. For demonstration, we will need at least the `mcp` functionality of Qwen-Agent.

In [ ]:
!pip3 install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
# `pip install -U qwen-agent` will install the minimal requirements.
# The optional requirements, specified in double brackets, are:
#   [gui] for Gradio-based GUI support;
#   [rag] for RAG support;
#   [code_interpreter] for Code Interpreter support;
#   [mcp] for MCP support.

# Step 1: Create an Agent

Create an agent that is capable of utilizing map functionalities provided by Amap.

An api key for Amap is needed, instructions are at https://lbs.amap.com/api/mcp-server/summary

In [ ]:
from qwen_agent.agents import Assistant
from qwen_agent.utils.output_beautify import typewriter_print
# `typewriter_print` 以非重叠的方式打印流式消息，便于清晰查看。

In [5]:
import dotenv
# Load environment variables from a .env file if it exists
dotenv.load_dotenv()
DASHSCOPE_API_KEY = dotenv.get_key(dotenv.find_dotenv(), 'DASHSCOPE_API_KEY')
AMAP_API_KEY = dotenv.get_key(dotenv.find_dotenv(), 'AMAP_API_KEY')

In [6]:
llm_cfg = {
    'model': 'qwen3-32b',
    'model_server': 'dashscope',
    'api_key': DASHSCOPE_API_KEY # **fill your dashscope api key here**

    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    # 'model': 'Qwen3-8B',
    # 'model_server': 'http://localhost:8000/v1',  # base_url, also known as api_base
    # 'api_key': 'EMPTY'
}

tools = [
    {
        "mcpServers": {
            # enumeration of mcp server configs
            "amap-amap-sse": {
                "url": "https://mcp.amap.com/sse?key="+AMAP_API_KEY  # **fill your amap mcp key**
            }
        }
    }
]

agent = Assistant(
    llm=llm_cfg, 
    function_list=tools
)

2025-06-04 10:50:33,738 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['amap-amap-sse']
2025-06-04 10:50:33,739 - mcp_manager.py - 338 - INFO - amap-amap-sse sse_read_timeout: 300s


Error in sse_reader: 


# Step 2: Calculate the Distance

We can query the distance between two locations by prompting the agent, which will then provide an answer utilizing the MCP server.

In [7]:
query = "How far is it between Zhejiang University (more specifically, the Zijingang Campus) and Hangzhou Xiaoshan airport?"
messages = [{'role': 'user', 'content': query}]

In [8]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user is asking for the distance between Zhejiang University's Zijingang Campus and Hangzhou Xiaoshan Airport. Let me figure out how to approach this.

First, I need the coordinates of both locations. The user mentioned specific places, so I should use the geocoding function to get their latitudes and longitudes. The function amap-amap-sse-maps_geo can convert addresses to coordinates. 

For Zhejiang University Zijingang Campus, the address would be something like "Zijingang Campus, Zhejiang University, Hangzhou, China". Similarly, Hangzhou Xiaoshan Airport is a well-known landmark, so the address is straightforward. 

Once I have both coordinates, I can use the distance measurement function amap-amap-sse-maps_distance. The user didn't specify the type of distance (driving, walking, straight line), but since it's between a campus and an airport, driving distance is probably most relevant. The type parameter would be 1 for driving. 

Wait, the user might want the straig

In [9]:
messages += ret_messages # extending the context with new `ret_messages`.

# Step 3: Make a Drive Plan

We can create a drive plan by asking follow-up questions, and the agent will generate the plan for us.

In [10]:
messages += [{'role': 'user', 'content': 'Okay, how can i drive?'}]

In [11]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user just asked, "Okay, how can I drive?" after I provided the distance and estimated time between Zhejiang University's Zijingang Campus and Hangzhou Xiaoshan Airport. Let me break this down.

First, the user is interested in driving directions now. They already know the distance and time, so the next step is to get the actual route. I need to figure out which function to use for driving directions. Looking at the tools provided, there's the "amap-amap-sse-maps_direction_driving" function. That's the one for驾车路径规划, which translates to driving route planning. The parameters required are origin and destination coordinates. 

I remember from the previous interaction that I used the amap-amap-sse-maps_geo function to get the coordinates for both locations. The Zijingang Campus was at 120.088667,30.304176 and the airport is at 120.427708,30.228153. So those coordinates should be used as the origin and destination in the driving directions function.

Wait, the user might n

# Use the GUI

We have explored the capabilities of the Qwen-Agent framework and Qwen models in providing travel suggestions. We can also achieve this by using the GUI.

In [8]:
from qwen_agent.gui import WebUI

agent = Assistant(
    name="Qwen Guide",
    description="I can help with your travel needs, ask me anything!",
    llm=llm_cfg,
    function_list=tools
)

WebUI(agent).run()

2025-05-01 21:14:00,386 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['amap-amap-sse']


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
